In [1]:
# import packages
%matplotlib inline
import numpy as np
import math
np.math = math  # Redirect numpy.math to the built-in math module

import denoising
import importlib
denoising = importlib.reload(denoising)
import utils
utils = importlib.reload(utils)

use torch backend


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import h, k, c
from astropy import units as u

# Load the column density map from file
logN_H = np.load('Archive/Turb_3.npy')[1]  
# logN_H = utils.downsample_by_four(logN_H)
nx, ny = logN_H.shape

# Define constant dust temperature
T_d = 10  # Typical Planck dust temperature in K

# int_val = []
# for nu in nu_list:
#     int_val.append(np.mean(modified_blackbody(logN_H, T_d, nu*1e9)))
# int_val = np.array(int_val)

# Observation frequency (e.g., 353 GHz in Hz)

# Compute the mock observed intensity map in μK_CMB
nu = (217,353)
I_nu_map_μK_nu1 = utils.modified_blackbody(logN_H, T_d, nu[0]*1e9)
I_nu_map_μK_nu2 = utils.modified_blackbody(logN_H, T_d, nu[1]*1e9)

In [3]:
# Ensure inputs have shape (1, H, W)
dust_nu1 = I_nu_map_μK_nu1[None, :, :]
dust_nu2 = I_nu_map_μK_nu2[None, :, :]

# --- Parameters ---
n_realizations = 10
SNR = 1
amplitude = 2.
spectral_index = -1.7

nx, ny = dust_nu1.shape[-2], dust_nu1.shape[-1]

# --- Compute noise variances ---
variance_nu1 = (np.std(dust_nu1) / SNR) ** 2
variance_nu2 = (np.std(dust_nu2) / SNR) ** 2
variance = (variance_nu1, variance_nu2)

# --- Create contamination_arr with shape (n_realizations, 2, 1, H, W) ---
contamination_arr = np.zeros((n_realizations, 2, 1, nx, ny), dtype=np.float32)

for i in range(n_realizations):
    # Shared CMB: shape (1, H, W)
    cmb_map = utils.generate_cmb_map(n_x=nx, n_y=ny, amplitude=amplitude, spectral_index=spectral_index)
    cmb_map = cmb_map.cpu().numpy()[None, :, :]

    # Independent noise: shape (1, H, W)
    noise_nu1 = np.random.normal(0, np.sqrt(variance_nu1), (1, nx, ny))
    noise_nu2 = np.random.normal(0, np.sqrt(variance_nu2), (1, nx, ny))

    # Total contamination: shape (1, H, W)
    contamination_arr[i, 0] = noise_nu1 + cmb_map
    contamination_arr[i, 1] = noise_nu2 + cmb_map

contamination_arr_nu1 = contamination_arr[:, 0]  # shape: (Mn, 1, H, W)
contamination_arr_nu2 = contamination_arr[:, 1]  # shape: (Mn, 1, H, W)


In [7]:
SNR = 1
variance_nu1 = (np.std(dust_nu1)/SNR)**2
variance_nu2 = (np.std(dust_nu2)/SNR)**2
variance = (variance_nu1, variance_nu2)

noise_nu1 = np.random.normal(0, np.sqrt(variance_nu1), dust_nu1.shape)
noise_nu2 = np.random.normal(0, np.sqrt(variance_nu2), dust_nu2.shape)

cmb_map = utils.generate_cmb_map(n_x=nx, n_y=ny, amplitude=amplitude, spectral_index=spectral_index)
cmb_map = cmb_map.cpu().numpy()[None, :, :]

data_nu1 = dust_nu1 + noise_nu1 + cmb_map
data_nu2 = dust_nu2 + noise_nu2 + cmb_map

# define target
image_target_nu1 = data_nu1 
image_target_nu2 = data_nu2 

# definte initial maps for optimisation
image_init_nu1 = image_target_nu1
image_init_nu2 = image_target_nu2

estimator_name = 's_cov'
std_nu1 = denoising.compute_std(image_target_nu1, contamination_arr = contamination_arr_nu1) 
std_nu2 = denoising.compute_std(image_target_nu2, contamination_arr = contamination_arr_nu2) 
std_double = denoising.compute_std_double(image_target_nu1, image_target_nu2, contamination_arr = contamination_arr, image_ref1=image_target_nu1, image_ref2 = image_target_nu2,  wavelets='BS') 
std = (std_nu1, std_nu2)

In [8]:
n_epochs = 3 #number of epochs
# decontaminate
for i in range(n_epochs):
    print(f'Starting epoch {i+1}')
    running_map = denoising.denoise_double(image_target_nu1, image_target_nu2, contamination_arr = contamination_arr, std = std, std_double=std_double, image_init1 = image_init_nu1, image_init2 = image_init_nu2, seed=0, print_each_step=True, steps = 25)

    std_nu1 = denoising.compute_std(running_map[0], contamination_arr = contamination_arr_nu1) 
    std_nu2 = denoising.compute_std(running_map[1], contamination_arr = contamination_arr_nu2) 
    std_double = denoising.compute_std_double(running_map[0], running_map[1], contamination_arr = contamination_arr, image_ref1=image_target_nu1, image_ref2 = image_target_nu2) 
    std = (std_nu1, std_nu2)

image_syn_nu1 = running_map[0]
image_syn_nu2 = running_map[1]

Starting epoch 1
Current Loss: 1.34e+01
Current Loss: 1.34e+01
Current Loss: 1.34e+01
Current Loss: 1.06e+01
Current Loss: 8.75e+00
Current Loss: 6.88e+00
Current Loss: 5.48e+00
Current Loss: 4.29e+00
Current Loss: 3.27e+00
Current Loss: 2.61e+00
Current Loss: 2.08e+00
Current Loss: 1.70e+00
Current Loss: 1.45e+00
Current Loss: 1.21e+00
Current Loss: 1.08e+00
Current Loss: 9.66e-01
Current Loss: 8.76e-01
Current Loss: 8.21e-01
Current Loss: 7.70e-01
Current Loss: 7.35e-01
Current Loss: 7.10e-01
Current Loss: 6.85e-01
Current Loss: 6.66e-01
Current Loss: 6.57e-01
Current Loss: 6.40e-01
Time used:  305.76549911499023 s
Starting epoch 2


/Users/tsouros/Desktop/ScatteringDenoising/denoising/FiltersSet.py:230: RuntimeWarning: divide by zero encountered in divide
  radial = np.exp(-(k - k0)**2 / (2*k*k0 - k**2)) #(k0**2 - (k-k0)**2)) #
/Users/tsouros/Desktop/ScatteringDenoising/denoising/FiltersSet.py:230: RuntimeWarning: overflow encountered in exp
  radial = np.exp(-(k - k0)**2 / (2*k*k0 - k**2)) #(k0**2 - (k-k0)**2)) #


Current Loss: 2.18e+01
Current Loss: 2.18e+01
Current Loss: 2.07e+01
Current Loss: 1.63e+01
Current Loss: 1.23e+01
Current Loss: 9.38e+00
Current Loss: 7.49e+00
Current Loss: 5.82e+00
Current Loss: 4.77e+00
Current Loss: 3.94e+00
Current Loss: 3.10e+00
Current Loss: 2.56e+00
Current Loss: 2.10e+00
Current Loss: 1.74e+00
Current Loss: 1.48e+00
Current Loss: 1.25e+00
Current Loss: 1.10e+00
Current Loss: 9.85e-01
Current Loss: 8.62e-01
Current Loss: 7.94e-01
Current Loss: 7.32e-01
Current Loss: 6.87e-01
Current Loss: 6.53e-01
Current Loss: 6.24e-01
Current Loss: 6.06e-01
Time used:  305.55189990997314 s
Starting epoch 3
Current Loss: 3.54e+01
Current Loss: 3.54e+01
Current Loss: 1.75e+01
Current Loss: 1.27e+01
Current Loss: 1.05e+01
Current Loss: 8.68e+00
Current Loss: 7.08e+00
Current Loss: 5.63e+00
Current Loss: 4.66e+00
Current Loss: 3.93e+00
Current Loss: 3.24e+00
Current Loss: 2.78e+00
Current Loss: 2.39e+00
Current Loss: 1.99e+00
Current Loss: 1.71e+00
Current Loss: 1.47e+00
Current

In [9]:
# Convert tuples to NumPy arrays
dust = np.stack([dust_nu1[0], dust_nu2[0]])  # Shape: (2, ...)
data = np.stack([data_nu1[0], data_nu2[0]])  # Shape: (2, ...)
image_denoised = np.stack([image_syn_nu1[0], image_syn_nu2[0]])  # Ensure it's an array

cmb = True
noise_optimisation = False
# Create an array of objects to preserve different shapes
results = np.array([dust, data, image_denoised])
np.save(f"nu={nu}_cmb={cmb}", results)